In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import sys

# Stream httpx/httpcore debug logging into the notebook output
formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(name)s: %(message)s")

for logger_name in ("httpx", "httpcore"):
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.DEBUG)
    if not any(getattr(handler, "name", "") == "httpx_debug" for handler in logger.handlers):
        handler = logging.StreamHandler(sys.stdout)
        handler.name = "httpx_debug"
        handler.setLevel(logging.DEBUG)
        handler.setFormatter(formatter)
        logger.addHandler(handler)


In [3]:
import atexit
from openai import OpenAI
import httpx

body_logger = logging.getLogger("httpx.body")
body_logger.setLevel(logging.DEBUG)

def log_post_payload(request: httpx.Request) -> None:
    if request.method.upper() != "POST":
        return
    content = request.content
    if not content:
        return
    try:
        payload = content.decode()
    except UnicodeDecodeError:
        payload = repr(content)
    body_logger.debug("POST %s payload: %s", request.url, payload)

http_client = httpx.Client(event_hooks={"request": [log_post_payload]})
atexit.register(http_client.close)
client = OpenAI(http_client=http_client)


In [4]:
m = client.models.list()

2025-10-05 21:11:21,263 [DEBUG] httpcore.connection: connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
2025-10-05 21:11:21,280 [DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x114c28560>
2025-10-05 21:11:21,280 [DEBUG] httpcore.connection: start_tls.started ssl_context=<ssl.SSLContext object at 0x114382950> server_hostname='api.openai.com' timeout=5.0
2025-10-05 21:11:21,293 [DEBUG] httpcore.connection: start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x114b7ce90>
2025-10-05 21:11:21,294 [DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'GET']>
2025-10-05 21:11:21,294 [DEBUG] httpcore.http11: send_request_headers.complete
2025-10-05 21:11:21,295 [DEBUG] httpcore.http11: send_request_body.started request=<Request [b'GET']>
2025-10-05 21:11:21,295 [DEBUG] httpcore.http11: send_request_body.complete
2025-10-05 21:11:21,295 [D

In [5]:
from interactive_json_tree import JSON
JSON(m.model_dump())

In [6]:
from pydantic import BaseModel
class Item(BaseModel):
    name: str
    price: str

class Menu(BaseModel):
    restaurant: str
    items: list[Item]


In [7]:
response = client.responses.parse(
    model="gpt-5-nano",
    input="What appetizers available at Kava on shawmut ave ?",
    tools=[{"type": "web_search"}],
    text_format=Menu,
)


2025-10-05 21:11:27,157 [DEBUG] httpx.body: POST https://api.openai.com/v1/responses payload: {"input":"What appetizers available at Kava on shawmut ave ?","model":"gpt-5-nano","text":{"format":{"type":"json_schema","strict":true,"name":"Menu","schema":{"$defs":{"Item":{"properties":{"name":{"title":"Name","type":"string"},"price":{"title":"Price","type":"string"}},"required":["name","price"],"title":"Item","type":"object","additionalProperties":false}},"properties":{"restaurant":{"title":"Restaurant","type":"string"},"items":{"items":{"$ref":"#/$defs/Item"},"title":"Items","type":"array"}},"required":["restaurant","items"],"title":"Menu","type":"object","additionalProperties":false}}},"tools":[{"type":"web_search"}]}
2025-10-05 21:11:27,157 [DEBUG] httpcore.connection: close.started
2025-10-05 21:11:27,158 [DEBUG] httpcore.connection: close.complete
2025-10-05 21:11:27,158 [DEBUG] httpcore.connection: connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 so

In [ ]:
JSON(response.model_dump())

TODO
----------
- Streaming: log messages, replay, visualize
- converations, other parameters
- log api call -- using httpx logger